In [ ]:
!pip install rapidfuzz
from google.colab import drive


!pip install python-Levenshtein
import Levenshtein


drive.mount('/content/drive')


import pandas as pd
from scipy.spatial.distance import pdist
from rapidfuzz import process, fuzz
import numpy as np
from scipy.stats import norm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.2 MB 7.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 174 kB 6.3 MB/s 


In [ ]:
# Get data, create usable format
movie_pivot = pd.read_csv('/content/drive/MyDrive/movie_pivot.csv', index_col=[0])
movie_meta = pd.read_csv('/content/drive/MyDrive/movie_meta.csv', index_col=[0])

valid_movies = movie_pivot.index.intersection(movie_meta.index)

movie_pivot = movie_pivot.loc[valid_movies]
movie_meta = movie_meta.loc[valid_movies]

# Get data, create usable format
book_pivot = pd.read_csv('/content/drive/MyDrive/book_pivot.csv', index_col=[0])
book_meta = pd.read_csv('/content/drive/MyDrive/book_meta.csv', index_col=[0])

# # Create frame for storing distances
movie_dist = pd.DataFrame(index=movie_pivot.index, data=movie_meta[['title']])
book_dist = pd.DataFrame(index=book_meta.index, data=book_meta[['title']])


# allow interfacing
shared_labels = list(set(movie_pivot.columns).intersection(set(book_pivot.columns)))

book_pivot = book_pivot[list(shared_labels)]
movie_pivot = movie_pivot[list(shared_labels)]

In [ ]:
# Function to search movie name, returns index in database
def search_movies(name, index):
    dist = []

    for t in movie_meta['title']:
        dist.append(Levenshtein.distance(t, name, weights=(100, 1, 10)))

    df = pd.DataFrame(movie_meta[['title']])
    df['dist'] = dist
    selection = df.sort_values('dist', ascending=True).head(5).reset_index()

    display(selection[['title']])
    print("Selection:", index)

    return selection['item_id'][index]

# Function to search movie name, returns index in database
def search_books(name, index):
    dist = []

    for t in book_meta['title']:
        dist.append(Levenshtein.distance(t.lower(), name.lower(), weights=(100, 1, 10)))

    df = pd.DataFrame(book_meta[['title']])
    df['dist'] = dist
    selection = df.sort_values('dist', ascending=True).head(5).reset_index()

    display(selection[['title']])
    print("Selection:", index)

    return selection['item_id'][index]

In [ ]:
# book to recommended

base_id = search_books('Twilight #1', 2)

movie_dist['Dist_1'] = (movie_pivot * book_pivot.loc[base_id]).sum(axis=1).to_list()
book_dist['Dist_1'] = (book_pivot * book_pivot.loc[base_id]).sum(axis=1).to_list()

In [ ]:
book_pivot.loc[base_id]

In [ ]:
movie_dist.sort_values("Dist_1", ascending=False).head(10)

In [ ]:
book_dist.sort_values("Dist_1", ascending=False).head(10)

In [ ]:
#take list of book ids, create recommendation for a book

target_list = book_dist.sort_values('Dist_1', ascending=False).head(5).index # replace with a [list of ids]

target = book_pivot.loc[target_list].mean()

movie_dist['Dist_1'] = (movie_pivot * target * np.log(book_pivot.loc[target_list].var().sort_values().max() / book_pivot.loc[target_list].var().sort_values())).sum(axis=1).to_list()
movie_dist.sort_values("Dist_1", ascending=False).head(10)